## Preparing data for Word2Vec and exporting as a dataset to use in Word2vec

Using this approach: https://muhark.github.io/python/ml/nlp/2021/10/21/word2vec-from-scratch.html

In [124]:
import sys

!{sys.executable} -m pip  install --quiet torch duckdb duckdb-engine ipython-sql sqlalchemy python-snappy pyarrow memray pandas watermark ipywidgets bs4 polars rich humanize matplotlib

DEPRECATION: Configuring installation scheme with distutils config files is deprecated and will no longer work in the near future. If you are using a Homebrew or Linuxbrew Python, please see discussion at https://github.com/Homebrew/homebrew-core/issues/76621
  DEPRECATION: Configuring installation scheme with distutils config files is deprecated and will no longer work in the near future. If you are using a Homebrew or Linuxbrew Python, please see discussion at https://github.com/Homebrew/homebrew-core/issues/76621
DEPRECATION: Configuring installation scheme with distutils config files is deprecated and will no longer work in the near future. If you are using a Homebrew or Linuxbrew Python, please see discussion at https://github.com/Homebrew/homebrew-core/issues/76621
You should consider upgrading via the '/usr/local/opt/python@3.9/bin/python3.9 -m pip install --upgrade pip' command.


In [1]:
%load_ext sql

In [2]:
import pandas as pd
import json

# display entire row
pd.set_option("display.max_colwidth", None)
pd.set_option("display.max_columns", None)

In [3]:
# DuckDB Magics
%config SqlMagic.autopandas = True
%config SqlMagic.feedback = False
%config SqlMagic.displaycon = False

In [4]:
# Connect to DuckDB
%sql duckdb:///my-db.duckdb

In [ ]:
# Create parquet file
# Use bdt tool https://github.com/andygrove/bdt/blob/main/src/parquet.rs
# bdt convert goodreads_books.json goodreads_parquet.parquet

In [490]:
# Create from parquet file
%sql CREATE TABLE books_pqt AS SELECT * FROM '/Users/vicki/viberary/viberary/notebooks/goodreads_parquet.parquet/*.parquet';

# create from CSV
%sql CREATE TABLE goodreads_schema AS SELECT * FROM '/Users/vicki/viberary/viberary/notebooks/goodreads.csv';

,Count
0,2360655


In [83]:
# create from CSV with schema

%sql CREATE TABLE goodreads_schema AS SELECT * FROM read_csv('/Users/vicki/viberary/viberary/notebooks/goodreads.csv', header=True, columns={'index_col': 'INTEGER', 'asin': 'VARCHAR','authors': 'VARCHAR','average_rating': 'DOUBLE','book_id': 'INTEGER','country_code': 'VARCHAR','description': 'VARCHAR','edition_information': 'VARCHAR','format': 'VARCHAR','image_url': 'VARCHAR','is_ebook': 'VARCHAR','isbn': 'VARCHAR','isbn13': 'VARCHAR','kindle_asin': 'VARCHAR', 'language_code': 'VARCHAR', 'link': 'VARCHAR', 'num_pages': 'INTEGER','popular_shelves': 'STRING[]','publication_day': 'INTEGER','publication_mon': 'INTEGER','publication_year': 'INTEGER','publisher': 'VARCHAR','ratings_count': 'FLOAT','series': 'VARCHAR','similar_books': 'VARCHAR','text_reviews_count': 'VARCHAR','title': 'VARCHAR','title_without_series': 'VARCHAR','url': 'VARCHAR', 'work_id': 'VARCHAR'});;

""


In [84]:
%sql describe goodreads_schema

,column_name,column_type,null,key,default,extra
0,index_col,INTEGER,YES,None,None,None
1,asin,VARCHAR,YES,None,None,None
2,authors,VARCHAR,YES,None,None,None
3,average_rating,DOUBLE,YES,None,None,None
4,book_id,INTEGER,YES,None,None,None
5,country_code,VARCHAR,YES,None,None,None
6,description,VARCHAR,YES,None,None,None
7,edition_information,VARCHAR,YES,None,None,None
8,format,VARCHAR,YES,None,None,None
9,image_url,VARCHAR,YES,None,None,None


In [19]:
# Inspect data
%sql  select title, description, REGEXP_REPLACE(popular_shelves, '(} {|}\n+)', '},{') from goodreads_schema LIMIT 10;

,title,description,"regexp_replace(popular_shelves, '(} {|}\n+)', '},{')"
0,W.C. Fields: A Life on Film,None,"[{'count': '3', 'name': 'to-read'},{'count': '1', 'name': 'p'}\n {'count': '1', 'name': 'collection'} {'count': '1', 'name': 'w-c-fields'}\n {'count': '1', 'name': 'biography'}]"
1,Good Harbor,"Anita Diamant's international bestseller ""The Red Tent"" brilliantly re-created the ancient world of womanhood. Diamant brings her remarkable storytelling skills to ""Good Harbor"" -- offering insight to the precarious balance of marriage and career, motherhood and friendship in the world of modern women. The seaside town of Gloucester, Massachusetts is a place where the smell of the ocean lingers in the air and the rocky coast glistens in the Atlantic sunshine. When longtime Gloucester-resident Kathleen Levine is diagnosed with breast cancer, her life is thrown into turmoil. Frightened and burdened by secrets, she meets Joyce Tabachnik -- a freelance writer with literary aspirations -- and a once-in-a-lifetime friendship is born. Joyce has just bought a small house in Gloucester, where she hopes to write as well as vacation with her family. Like Kathleen, Joyce is at a fragile place in her life.\nA mutual love for books, humor, and the beauty of the natural world brings the two women together. They share their personal histories, and help each other to confront scars left by old emotional wounds.\nWith her own trademark wisdom and humor, Diamant considers the nature, strength, and necessity of adult female friendship. ""Good Harbor"" examines the tragedy of loss, the insidious nature of family secrets, as well as the redemptive power of friendship.","[{'count': '2634', 'name': 'to-read'},{'count': '160', 'name': 'fiction'}\n {'count': '92', 'name': 'currently-reading'}\n {'count': '25', 'name': 'contemporary-fiction'}\n {'count': '22', 'name': 'owned'} {'count': '19', 'name': 'book-club'}\n {'count': '17', 'name': 'chick-lit'}\n {'count': '16', 'name': 'friendship'}\n {'count': '15', 'name': 'books-i-own'} {'count': '12', 'name': 'novel'}\n {'count': '12', 'name': 'general-fiction'}\n {'count': '11', 'name': 'anita-diamant'}\n {'count': '10', 'name': 'owned-books'}\n {'count': '10', 'name': 'contemporary'} {'count': '10', 'name': 'women'}\n {'count': '8', 'name': 'adult'} {'count': '7', 'name': 'default'}\n {'count': '7', 'name': 'favorites'}\n {'count': '7', 'name': 'didn-t-finish'} {'count': '6', 'name': 'own-it'}\n {'count': '6', 'name': 'library'} {'count': '6', 'name': 'new-england'}\n {'count': '6', 'name': 'bookclub'}\n {'count': '6', 'name': 'women-s-fiction'}\n {'count': '5', 'name': 'breast-cancer'}\n {'count': '5', 'name': 'beach-ocean'}\n {'count': '5', 'name': 'audio-book'}\n {'count': '5', 'name': 'historical-fiction'}\n {'count': '5', 'name': 'my-library'} {'count': '5', 'name': '2006'}\n {'count': '5', 'name': 'literature'} {'count': '5', 'name': 'audiobook'}\n {'count': '4', 'name': 'read-2015'} {'count': '4', 'name': 'diamant'}\n {'count': '4', 'name': 'audio-books'}\n {'count': '4', 'name': 'diamant-anita'}\n {'count': '4', 'name': 'adult-fiction'}\n {'count': '4', 'name': 'my-books'} {'count': '4', 'name': 'jewish'}\n {'count': '4', 'name': 'novels'} {'count': '4', 'name': 'library-book'}\n {'count': '4', 'name': '2002'} {'count': '3', 'name': 'read-2016'}\n {'count': '3', 'name': 'relationships'}\n {'count': '3', 'name': 'female-friendship'}\n {'count': '3', 'name': 'favorite-authors'}\n {'count': '3', 'name': 'kindle'} {'count': '3', 'name': 'have'}\n {'count': '3', 'name': 'strong-women'} {'count': '3', 'name': 'cancer'}\n {'count': '3', 'name': 'own-to-read'}\n {'count': '3', 'name': 'audiobooks'} {'count': '3', 'name': 'read-2011'}\n {'count': '3', 'name': 'audio'} {'count': '3', 'name': 'drama'}\n {'count': '3', 'name': 'read-2009'} {'count': '3', 'name': 'i-own'}\n {'count': '3', 'name': 'book-group'} {'count': '3', 'name': 'american'}\n {'count': '3', 'name': 'fiction-general'}\n {'count': '3', 'nam

In [66]:
text_data["popular_shelves"] = text_data["popular_shelves"].replace(regex="'", value='"')

In [67]:
exploded = (
    pd.concat({i: pd.DataFrame(x) for i, x in text_data.pop("popular_shelves").items()})
    .reset_index(level=1, drop=True)
    .join(text_data)
    .reset_index(drop=True)
)

KeyboardInterrupt: 

In [59]:
# Description field to string
exploded["description_string"] = exploded.description.astype(str)
exploded["description_string"] = exploded.description_string.replace(
    "None", "", regex=True
)

In [61]:
# Count of popular shelves tags to int
exploded["count"] = exploded["count"].astype(int)
exploded["weighted_count"] = exploded["count"] * (exploded["name"] + " ")

In [62]:
# Let's start with a simple model using title and description and tags
exploded["word2vec_data"] = (
    (exploded["title"] + " ")
    + (exploded["description_string"] + " ")
    + exploded["weighted_count"]
)

In [63]:
# Include weighted tags in word2vec input
word2vec = (
    exploded.groupby(["title", "description"])["weighted_count"]
    .apply(",".join)
    .reset_index()
)
word2vec["word2vec_data"] = (
    (word2vec["description"] + " ")
    + (word2vec["title"] + " ")
    + word2vec["weighted_count"]
)
word2vec

title  \
0                All's Fairy in Love and War (Avalon: Web of Magic, #8)   
1                                                  Best Friends Forever   
2                                                           Good Harbor   
3                                     Playmaker: A Venom Series Novella   
4  Runic Astrology: Starcraft and Timekeeping in the Northern Tradition   
5                                         The Aeneid for Boys and Girls   
6                                                  The Devil's Notebook   
7                   The Unschooled Wizard (Sun Wolf and Starhawk, #1-2)   
8                                                 The Wanting of Levine   
9                                           W.C. Fields: A Life on Film   

                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                              description  \
0                                                                                                                                                                                                                                                                                                                                                                                                                                                           To Kara's astonishment, she discovers that a portal has opened in her bedroom closet and two goblins have fallen through! They refuse to return to the fairy realms and be drafted for an impending war. In an attempt to roust the pesky creatures, Kara falls through the portal, smack into the middle of a huge war. Kara meets Queen Selinda, who appoints Kara as a Fairy Princess and assigns her an impossible task: to put an end to the war using her diplomatic skills.\nAll's Fairy In Love And War is the eighth book in Avalon: Web of Magic, a twelve-book fantasy series for middle grade readers. Through their magical journey, the teenage heroines discover who they really are . . . and run into plenty of good guys, bad guys, and cute guys. Out of print for two years, Seven Seas is pleased to return the Avalon series to print in editions targeted for today's readers, with new manga-style covers and interior illustrations.   
1                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                             

In [64]:
# Preprocessing of data for word2vec

# Lower case
word2vec["word2vec_data"].str.lower()

# Remove everything except a-z@

# Split on spaces

# Remove stopwords

0                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                       

In [125]:
import torch